In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import re
import seaborn as sns
import statsmodels.stats.multitest
#import altair as alt
#alt.renderers.enable('notebook') #Necessary for Altair to work

In [2]:
import CPTAC

Loading CPTAC data:
Loading Dictionary...
Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter CPTAC.embargo() to open the webpage for more details.


In [4]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()
transcriptomics = CPTAC.get_transcriptomics()
transcriptomics_columns = CPTAC.get_transcriptomics_cols()

#print(transcriptomics_columns)
#print(transcriptomics)

Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A3GALT2', 'A4GALT', 'A4GNT',
       ...
       'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', name='idx', length=28057)


In [5]:
siteList = ['A1BG','ZXDB','ZYX']

In [6]:
p_values = []
gene = 'ARID1A'
for site in siteList:
    transdf = CPTAC.compare_mutations(transcriptomics, site, gene)
    print(transdf)


      A1BG           Mutation Patient_Type
S001  4.02  Missense_Mutation        Tumor
S002  4.81  Nonsense_Mutation        Tumor
S003  6.24           Wildtype        Tumor
S005  5.31           Wildtype        Tumor
S006  9.84           Wildtype        Tumor
S007  5.03           Wildtype        Tumor
S008  3.17    Frame_Shift_Del        Tumor
S009  6.03           Wildtype        Tumor
S010  6.02           Wildtype        Tumor
S011  5.71           Wildtype        Tumor
S012  5.17  Nonsense_Mutation        Tumor
S013  5.99           Wildtype        Tumor
S014  5.38  Nonsense_Mutation        Tumor
S015  5.95           Wildtype        Tumor
S016  5.86           Wildtype        Tumor
S017  3.27           Wildtype        Tumor
S018  4.65  Nonsense_Mutation        Tumor
S019  4.33    Frame_Shift_Ins        Tumor
S020  4.58           Wildtype        Tumor
S021  4.60           Wildtype        Tumor
S022  4.90  Missense_Mutation        Tumor
S023  4.03  Nonsense_Mutation        Tumor
S024  4.62 